In [2]:
# 오디오 장치 인덱스 출력
import pyaudio

audio = pyaudio.PyAudio()

print("Available audio input devices:")
for i in range(audio.get_device_count()):
    dev = audio.get_device_info_by_index(i)
    if dev['maxInputChannels'] > 0:
        print(f"Index {i}: {dev['name']}")

audio.terminate()


Available audio input devices:
Index 0: Microsoft Sound Mapper - Input
Index 1: IN 7-8(BEHRINGER X-AIR)
Index 2: IN 1-2(BEHRINGER X-AIR)
Index 3: IN 1-8(BEHRINGER X-AIR)
Index 4: IN 5-6(BEHRINGER X-AIR)
Index 5: IN 3-4(BEHRINGER X-AIR)
Index 12: 주 사운드 캡처 드라이버
Index 13: IN 7-8(BEHRINGER X-AIR)
Index 14: IN 1-2(BEHRINGER X-AIR)
Index 15: IN 1-8(BEHRINGER X-AIR)
Index 16: IN 5-6(BEHRINGER X-AIR)
Index 17: IN 3-4(BEHRINGER X-AIR)
Index 29: IN 1-2(BEHRINGER X-AIR)
Index 30: IN 1-8(BEHRINGER X-AIR)
Index 31: IN 5-6(BEHRINGER X-AIR)
Index 32: IN 3-4(BEHRINGER X-AIR)
Index 33: IN 7-8(BEHRINGER X-AIR)
Index 34: IN 5-6 (IN 5-6)
Index 38: IN 3-4 (IN 3-4)
Index 39: IN 7-8 (IN 7-8)
Index 42: IN 1-2 (IN 1-2)
Index 43: IN 1-8 (IN 1-8)


In [2]:
# 녹음 파일 만들기

import pyaudio
import wave
import keyboard
from pydub import AudioSegment

# 설정
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024
WAV_OUTPUT_FILENAME = "output.wav"
MP3_OUTPUT_FILENAME = "output.mp3"

def record_audio():
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        input_device_index=2,
                        frames_per_buffer=CHUNK)
    print("Recording... Press Enter to stop.")
    frames = []

    while True:
        data = stream.read(CHUNK)
        frames.append(data)
        if keyboard.is_pressed('enter'):
            break

    print("Recording finished.")
    
    stream.stop_stream()
    stream.close()
    audio.terminate()

    with wave.open(WAV_OUTPUT_FILENAME, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

if __name__ == "__main__":
    record_audio()

Recording... Press Enter to stop.
Recording finished.


In [58]:
import subprocess

def convert_wav_to_aac(input_wav, output_aac):
    # FFmpeg 실행 파일 경로 (경로를 실제 FFmpeg 경로로 바꿔주세요)
    ffmpeg_path = r"C:\ffmpeg-7.1.1-full_build\bin\ffmpeg.exe"
    
    # FFmpeg 명령어로 WAV를 AAC로 변환
    command = [
        ffmpeg_path, 
        '-i', input_wav,      # 입력 파일
        '-c:a', 'aac',        # AAC 코덱 사용
        '-b:a', '192k',       # 비트레이트 (192kbps, 필요에 따라 조정)
        output_aac            # 출력 파일
    ]
    
    # subprocess를 이용해 FFmpeg 명령어 실행
    try:
        subprocess.run(command, check=True)
        print(f"파일 변환 성공: {output_aac}")
    except subprocess.CalledProcessError as e:
        print(f"변환 중 오류 발생: {e}")

if __name__ == "__main__":
    input_wav = "./output.wav"  # 변환할 WAV 파일 경로
    output_aac = "output.aac"  # 출력할 AAC 파일 경로
    convert_wav_to_aac(input_wav, output_aac)

파일 변환 성공: output.aac


In [ ]:
import requests
import json

class ClovaSpeechClient:
    # Clova Speech invoke URL (앱 등록 시 발급받은 Invoke URL)
    invoke_url = 'https://clovaspeech-gw.ncloud.com/invokeURL'
    # Clova Speech secret key (앱 등록 시 발급받은 Secret Key)
    secret = 'secretKey'

    def req_upload(self, file, completion, callback=None, userdata=None, forbiddens=None, boostings=None,
                   wordAlignment=True, fullText=True, diarization=None, sed=None):
        request_body = {
            'language': 'ko-KR',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': {"enable": diarization},
            'sed': sed,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        print(json.dumps(request_body, ensure_ascii=False).encode('UTF-8'))
        files = {
            'media': open(file, 'rb'),
            'params': (None, json.dumps(request_body, ensure_ascii=False).encode('UTF-8'), 'application/json')
        }
        response = requests.post(headers=headers, url=self.invoke_url + '/recognizer/upload', files=files)
        return response

if __name__ == '__main__':
    res = ClovaSpeechClient().req_upload(file='./output.aac', completion='sync', diarization='false')
    print(res)
    
    

b'{"language": "ko-KR", "completion": "sync", "callback": null, "userdata": null, "wordAlignment": true, "fullText": true, "forbiddens": null, "boostings": null, "diarization": {"enable": "false"}, "sed": null}'


In [48]:
print(res.json().get("text"))

유명 보석상이 소유한 다이아몬드 목걸이가 도난당했습니다. 목걸이가 사라진 날 밤 보석상에는 2명의 직원이 있었고 그중 1명이 절도 혐의를 받고 있습니다.
